**Extract latest options (current state)**

In [6]:
import json
import pandas as pd
import ast
import glob
import os
from typing import List


def extract_options(data: List) -> pd.DataFrame:
    """
    Extract all options and all of their values from the latest commit to get the current state of the software project.

    :param data: list of configuration data from commit history
    :return: dataframe contain all options and the values from the latest commit
    """
    config_data = []
    latest_commit = data["commit_data"][-1]

    if not latest_commit["is_latest_commit"]:
        raise Exception("The latest commit is not the last commit in the history.")

    for config_file in latest_commit["network_data"]["config_file_data"]:
        for pair in config_file["pairs"]:
            config_data.append({
                "concept": config_file["concept"],
                "file_path": config_file["file_path"],
                "option": pair["option"],
                "value": pair["value"],
                "type": pair["type"],
                
            })

    df = pd.DataFrame(config_data)

    return df

project_dir = "../data/test_projects"
project_type = project_dir.split("/")[-1]
project_files = glob.glob(project_dir + "/*.json")

if not os.path.exists(f"../data/options/{project_type}"):
    os.makedirs(f"../data/options/{project_type}")

for project_file in project_files:
    print(f"Processing {project_file}")
    try:
        with open(project_file, "r", encoding="utf-8") as src:
            data = json.load(src)

        # Extract options from the latest commit
        df_options = extract_options(data=data)
        df_options.to_csv(f"../data/options/{project_type}/{data['project_name']}_options_latest.csv", index=False)
    except Exception as e:
        print(f"Error processing {project_file}: {e}")
        continue



Processing ../data/test_projects/piggymetrics.json
Processing ../data/test_projects/test-config-repo.json
